In [ ]:
import pandas as pd
import pyodbc
import numpy as np
import plotly.express as px
from datetime import datetime
import statsmodels.api as sm

pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30

def file_to_string(fileName):
    file = open(fileName,'r')
    string = file.read()
    file.close()
    return string.replace('$DATE$','2023-08-28')

def timestamp():
    return datetime.now().strftime('%Y-%m-%d %I:%M:%S %p')

def run_query(query):
    print("Query Start " + timestamp())
    cnxn = pyodbc.connect('DSN=edp-workbench-cshub', autocommit=True)
    df = pd.read_sql_query(query,cnxn)
    cnxn.close()
    print("Query End " + timestamp())
    return df
    
def add_conditionals(df):
    df = df.copy()
    df['Date']=pd.to_datetime(df['Date'])
    
    df['1st Reactive ExAsst Group'] = (df['Employee'].isin((
        581139, 569375, 575996, 547655, 573190, 573276,
        572909, 573192, 573585, 573573, 572815, 552121,
        576911, 572247, 364717, 426097, 554487, 575731,
        576565, 577073, 577246, 579162, 580895, 580911,
        581015, 581145, 581275
        ))).astype(int)
    
    df['2nd Reactive ExAsst Group'] = (df['Employee'].isin((
        548026, 548646, 579630, 550446, 569153, 578299
    ))).astype(int)

    df['Proactive ExAsst Group'] = (df['Employee'].isin((
        573190, 573276, 572909, 573192, 573585,
        573573, 572815, 552121, 576911, 572247
        ))).astype(int)

    df['After Reactive  ExAsst Launch']=(df['Date']>=datetime(2023,9,25)).astype(int)
    df['After Proactive ExAsst Launch']=(df['Date']>=datetime(2023,11,10)).astype(int)
    df['Has Proactive'] = df['Proactive ExAsst Group'] * df['After Proactive ExAsst Launch']
    
    df['Has Reactive'] = np.sign((
        df['After Reactive  ExAsst Launch'] * df['1st Reactive ExAsst Group']
        +
        df['After Proactive ExAsst Launch'] * df['2nd Reactive ExAsst Group']
        ))

    df['Test Group For This Period'] = np.select(condlist = [df['Has Proactive']==1,df['Has Reactive']==1], choicelist = ['Proactive','Reactive'],default='Control')
    df['Test Group After Launch'] = np.select(
        condlist = [
            df['Proactive ExAsst Group']==1,
            np.logical_and(
                np.logical_or(df['1st Reactive ExAsst Group']==1,df['2nd Reactive ExAsst Group']==1)
                ,df['Proactive ExAsst Group']==0)
            ],
        choicelist = ['Proactive','Reactive'],default='Control')

    df['Day'] = df['Date'].dt.weekday
    days = {0:'Monday',1:'Tuesday',2:'Wednesday',3:'Thursday',4:'Friday',5:'Saturday',6:'Sunday'}
    for day in days:
        df[days[day]]=(df['Day']==day).astype(int)
    df['Week']=df['Date'].dt.isocalendar().week
    df['Weeks Ago'] = df['Week'].astype(int).max() - df['Week'].astype(int)
    for week in df['Week'].unique():
        df['Week '+str(week)]= (df['Week']==week).astype(int)
    # for queue in df['TaskQueue'].unique():
    #     df[queue] = (df['TaskQueue']==queue).astype(int)
    df['Constant'] =1
    return df

def add_metrics(df):
    df = df.copy()
    df['AHT'] = df['Total_Resolution_Time']/df['Sessions']
    df['HERO'] = df['HERO_XR_Score']/df['HERO_XR_Count']
    df['SP100'] = df['Total_Accepted']/df['Total_Eligible']
    df['Transfers'] = df['Transfer_Count']/df['Transfer_Score']
    df['Constant'] =1
    return df

In [ ]:
df_daily = run_query(file_to_string('DailyMetrics.SQL'))
df_daily.head()

In [ ]:
metricList = ['Sessions','AHT','HERO','SP100']
metric_Inputs = ['Sessions','Total_Resolution_Time''HERO_XR_Score','HERO_XR_Count', 'Transfer_Score', 'Transfer_Count',
              'Helix_Searches', 'Helix_Sessions' 'Total_Eligible','Total_Accepted']
test_df = add_conditionals(df_daily)
test_df = add_metrics(test_df)

test_df = test_df.dropna()
test_df.info(verbose=False)
display(test_df.head(5))
for metric in metricList:
    display(sm.OLS(
        endog = test_df[metric],
        exog = test_df[['Has Proactive','Has Reactive','Weeks Ago','Constant']]
        ).fit().summary())
    px.box(test_df,y=metric,color='Test Group For This Period', title=metric).show()
    graph_df = pd.pivot_table(test_df[test_df['Test Group After Launch']!='Control'].copy(),values=metric,columns='Test Group After Launch',index='Date')
    px.line(graph_df,title=metric).show()
    del graph_df
del test_df, metricList, metric_Inputs

In [ ]:
test_df = add_conditionals(df_daily)
test_df = add_metrics(test_df)

test_df = test_df.dropna()
# display(test_df.head(5))

display(sm.OLS(
    endog = test_df['AHT'],
    exog = test_df[['Has Proactive','Weeks Ago','Constant']]
    ).fit().summary())
del test_df

In [95]:
df_sessions = run_query(file_to_string('AHT And Sales.SQL'))

Query Start 2023-12-05 12:44:56 PM


C:\Users\austen.moore\AppData\Local\Temp\ipykernel_14772\3531319944.py:24: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



Query End 2023-12-05 12:45:15 PM


In [139]:
test_df = add_conditionals(df_sessions.copy())


for queue in test_df['TaskQueue'].unique():
    test_df[queue] = (test_df['TaskQueue']==queue).astype(int)


display(sm.OLS(
    endog = test_df['Handle Time'],
    exog = test_df[[ 'Constant', 'Has Proactive',
                    'Offered', 'Accepted', 'Weeks Ago',
                    'VZN_MTS_Bundle_EN_2233', 'VZN_Onboarding_2628'
                     ]]
    ).fit().summary())

del test_df

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            Handle Time   R-squared:                       0.072
Model:                            OLS   Adj. R-squared:                  0.071
Method:                 Least Squares   F-statistic:                     223.8
Date:                Tue, 05 Dec 2023   Prob (F-statistic):          2.16e-276
Time:                        13:09:12   Log-Likelihood:            -1.4419e+05
No. Observations:               17435   AIC:                         2.884e+05
Df Residuals:                   17428   BIC:                         2.884e+05
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Constant                1119.8022     33.393     33.534      0.000    1054.349    1185.256
Has Proactive           -141.8607     24.869     -5.704      0.000    -190.607     -93.114
Offered                  294.8686     14.946     19.729      0.000     265.573     324.165
Accepted                 699.8320     29.431     23.779      0.000     642.144     757.520
Weeks Ago                -19.3751      2.986     -6.489      0.000     -25.228     -13.522
VZN_MTS_Bundle_EN_2233   -85.3605     28.156     -3.032      0.002    -140.549     -30.172
VZN_Onboarding_2628       90.9916     33.650      2.704      0.007      25.034     156.949
==============================================================================
Omnibus:                     9179.172   Durbin-Watson:                   1.966
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            97903.726
Skew:                           2.312   Prob(JB):                         0.00
Kurtosis:                      13.649   Cond. No.                         42.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""